In [169]:
import pandas as pd


In [177]:
# Cargar tus DataFrames
df_marca = pd.read_csv('tables/marcas.csv')
df_modelo = pd.read_csv('tables/modelos.csv')
df_autos = pd.read_csv('tables/autos.csv')

In [178]:
df_marca.head()

,ID_MARCA,MARCA
0,1,TOYOTA
1,2,CHEVROLET
2,3,NISSAN
3,4,HYUNDAI
4,5,KIA


In [179]:
df_autos.head()

,MARCA,MODELO,CILINDRADA,AÑO,VERSION
0,TOYOTA,COROLLA,1.6,2005.0,NaN
1,TOYOTA,COROLLA,1.6,2006.0,NaN
2,TOYOTA,COROLLA,1.6,2007.0,NaN
3,TOYOTA,COROLLA,1.6,2008.0,NaN
4,TOYOTA,COROLLA,1.6,2009.0,NaN


In [180]:
df_modelo.head()

,ID_MODELO,ID_MARCA,MODELO
0,1,1,COROLLA
1,2,2,N300
2,3,3,TIIDA
3,4,3,NAVARA
4,5,4,ACCENT


In [171]:
import random 

def abreviar_marca(marca, n_variantes=10):
    marca = marca.upper().replace("-", " ")
    tokens = marca.split()
    variantes = set()

    if len(tokens) == 0:
        return []

    p = tokens[0]

    # Cortes más flexibles (3 a 6 letras)
    for i in range(2, min(7, len(p) + 1)):
        variantes.add(p[:i])

    # Solo consonantes
    consonantes = "".join([c for c in p if c not in "AEIOU"])
    if consonantes:
        variantes.add(consonantes[:3])

    # Palabra completa
    variantes.add(p)

    # Si hay segunda palabra, combinaciones
    if len(tokens) > 1:
        s = tokens[1]
        variantes.update({
            f"{p[:3]} {s}",
            f"{p[0]}{s}",
            s,
            f"{p} {s}",
            f"{p[:2]} {s[:2]}",
            f"{p[:2]}{s[:1]}"
        })

    # Todo junto sin espacios
    variantes.add("".join(tokens))

    return list(variantes)[:n_variantes]



def abreviar_modelo(modelo, n_variantes=5):
    modelo = modelo.upper().replace("-", " ")
    tokens = modelo.split()
    variantes = set()

    p = tokens[0]
    variantes.add(p[:3])
    variantes.add("".join([c for c in p if c not in "AEIOU"]))
    variantes.add(p)
    variantes.add(p[:2])
    
    if len(tokens) > 1:
        s = tokens[1]
        variantes.add(f"{p[:3]} {s}")
        variantes.add(f"{p[0]}{s}")
        variantes.add(s)
        variantes.add(f"{p} {s}")
        if len(s) > 1:
            variantes.add(f"{p[:2]} {s[:2]}")
        variantes.add(f"{p[:2]}{s[:1]}")

    variantes.add("".join(tokens))
    return list(variantes)




In [172]:
ABREVIACIONES_MARCAS = {}
for marca in df_marca['MARCA']:
    abreviaciones = abreviar_marca(marca, n_variantes=10)
    for abrev in abreviaciones:
        ABREVIACIONES_MARCAS[abrev] = marca

ABREVIACIONES_MODELOS = {}
for modelo in df_modelo['MODELO']:
    abreviaciones = abreviar_modelo(modelo, n_variantes=10)
    for abrev in abreviaciones:
        ABREVIACIONES_MODELOS[abrev] = modelo



In [173]:
import pandas as pd
from difflib import get_close_matches

# Función para normalizar texto
def normalizar_texto(texto):
    """Normaliza texto para comparación"""
    if pd.isna(texto) or texto is None:
        return ""
    texto = str(texto).upper().strip()
    reemplazos = {'Á':'A', 'É':'E', 'Í':'I', 'Ó':'O', 'Ú':'U', '-':' ', '_':' '}
    for orig, repl in reemplazos.items():
        texto = texto.replace(orig, repl)
    return ' '.join(texto.split())  # Elimina espacios múltiples

# Búsqueda de coincidencias aproximadas
def encontrar_coincidencia(valor, opciones, umbral=0.85):
    """Encuentra la mejor coincidencia aproximada"""
    opciones = [str(x) for x in opciones if not pd.isna(x)]
    coincidencias = get_close_matches(valor, opciones, n=1, cutoff=umbral)
    return coincidencias[0] if coincidencias else None

# Función principal
def verificar_auto_nuevo(nuevo_auto, df_marca, df_modelo, df_autos):
    """
    Verifica si un auto existe en la base de datos con manejo de errores.
    
    Args:
        nuevo_auto: Dict o lista con {'marca', 'modelo', 'cilindrada', 'año', 'version'}
        df_marca: DataFrame de marcas
        df_modelo: DataFrame de modelos
        df_autos: DataFrame de autos
    
    Returns:
        Dict con {'existe', 'mensaje', 'coincidencias', 'auto_normalizado'}
    """
    try:
        # Convertir lista a dict si es necesario
        if isinstance(nuevo_auto, list):
            if len(nuevo_auto) >= 2:
                nuevo_auto = {
                    'marca': nuevo_auto[0] if len(nuevo_auto) > 0 else '',
                    'modelo': nuevo_auto[1] if len(nuevo_auto) > 1 else '',
                    'cilindrada': nuevo_auto[2] if len(nuevo_auto) > 2 else '',
                    'año': nuevo_auto[3] if len(nuevo_auto) > 3 else None,
                    'version': nuevo_auto[4] if len(nuevo_auto) > 4 else None
                }
            else:
                return {
                    'existe': False,
                    'mensaje': 'Formato incorrecto: Se necesita al menos marca y modelo',
                    'coincidencias': [],
                    'auto_normalizado': None
                }
        elif not isinstance(nuevo_auto, dict):
            return {
                'existe': False,
                'mensaje': 'Formato incorrecto: Se espera un diccionario o lista',
                'coincidencias': [],
                'auto_normalizado': None
            }

        # Verificación de campos mínimos
        if 'marca' not in nuevo_auto or 'modelo' not in nuevo_auto:
            return {
                'existe': False,
                'mensaje': 'Faltan campos obligatorios: marca y modelo',
                'coincidencias': [],
                'auto_normalizado': None
            }

        # Normalizar campos de entrada
        marca_input = normalizar_texto(nuevo_auto.get('marca'))
        modelo_input = normalizar_texto(nuevo_auto.get('modelo'))
        cilindrada_input = normalizar_texto(nuevo_auto.get('cilindrada', ''))
        año_input = nuevo_auto.get('año')
        version_input = normalizar_texto(nuevo_auto.get('version', ''))
        
        # Paso 1: Normalizar marca
        marca_norm = ABREVIACIONES_MARCAS.get(marca_input, marca_input)
        if marca_norm not in df_marca['MARCA'].values:
            marca_coincidente = encontrar_coincidencia(marca_norm, df_marca['MARCA'])
            if marca_coincidente:
                marca_norm = marca_coincidente
            else:
                return {
                    'existe': False,
                    'mensaje': 'Marca no reconocida',
                    'coincidencias': [],
                    'auto_normalizado': None
                }
        
        # Paso 2: ID_MARCA
        id_marca = df_marca[df_marca['MARCA'] == marca_norm]['ID_MARCA'].values[0]
        
        # Paso 3: Coincidencia modelo
        modelos_disponibles = df_modelo[df_modelo['ID_MARCA'] == id_marca]['MODELO']
        modelo_norm = encontrar_coincidencia(modelo_input, modelos_disponibles)
        if not modelo_norm:
            return {
                'existe': False,
                'mensaje': f'Modelo no encontrado para {marca_norm}',
                'coincidencias': df_modelo[df_modelo['ID_MARCA'] == id_marca].to_dict('records'),
                'auto_normalizado': None
            }
        
        # Paso 4: Coincidencias exactas
        query = df_autos[
            (df_autos['MARCA'] == marca_norm) & 
            (df_autos['MODELO'] == modelo_norm)
        ].copy()
        
        if cilindrada_input:
            query = query[query['CILINDRADA'].apply(normalizar_texto) == cilindrada_input]
        if año_input is not None and str(año_input).strip() != "":
            try:
                año_input = int(float(año_input))  # Normaliza cosas como "2005.0" o "2005"
                query = query[query['AÑO'].apply(lambda x: int(float(x)) if pd.notna(x) else -1) == año_input]
            except:
                pass  # Por si el dato no se puede convertir

        if version_input.strip() != "":
            # Comparación normal
            query = query[
                query['VERSION'].fillna('').apply(normalizar_texto) == version_input
            ]
        else:
            # Si input vacío, buscar VERSION vacía o NaN
            query = query[
                query['VERSION'].isna() | 
                (query['VERSION'].fillna('').apply(normalizar_texto) == '')
            ]
        
        if not query.empty:
            return {
                'existe': True,
                'mensaje': 'Auto existe en la base de datos',
                'coincidencias': query.to_dict('records'),
                'auto_normalizado': {
                    'marca': marca_norm,
                    'modelo': modelo_norm,
                    'cilindrada': cilindrada_input,
                    'año': año_input,
                    'version': version_input
                }
            }

        # Paso 5: Coincidencias similares (sin usar ID_MODELO)
        coincidencias = df_autos[
            (df_autos['MARCA'] == marca_norm) & 
            (df_autos['MODELO'] == modelo_norm)
        ].to_dict('records')

        return {
            'existe': False,
            'mensaje': 'Auto no encontrado exactamente, pero hay modelos similares',
            'coincidencias': coincidencias,
            'auto_normalizado': {
                'marca': marca_norm,
                'modelo': modelo_norm,
                'cilindrada': cilindrada_input,
                'año': año_input,
                'version': version_input
            }
        }

    except Exception as e:
        return {
            'existe': False,
            'mensaje': f'Error: {str(e)}',
            'coincidencias': [],
            'auto_normalizado': None
        }




In [174]:
# Para ejecutar la interfaz de usuario
#interfaz_verificacion_auto(df_marca, df_modelo, df_autos)

# O para usar programáticamente
nuevo_auto = {
    "marca": "mitsub",
    "modelo": "Lancer",
    "cilindrada": "1.6",
    "año": "2010"
}
resultado = verificar_auto_nuevo(nuevo_auto, df_marca, df_modelo, df_autos)
resultado

{'existe': False,
 'mensaje': 'Auto no encontrado exactamente, pero hay modelos similares',
 'coincidencias': [{'MARCA': 'MITSUBISHI',
   'MODELO': 'LANCER',
   'CILINDRADA': '1.5',
   'AÑO': 1998.0,
   'VERSION': nan},
  {'MARCA': 'MITSUBISHI',
   'MODELO': 'LANCER',
   'CILINDRADA': '1.3',
   'AÑO': 1990.0,
   'VERSION': nan},
  {'MARCA': 'MITSUBISHI',
   'MODELO': 'LANCER',
   'CILINDRADA': '1.3',
   'AÑO': 1991.0,
   'VERSION': nan},
  {'MARCA': 'MITSUBISHI',
   'MODELO': 'LANCER',
   'CILINDRADA': nan,
   'AÑO': 2008.0,
   'VERSION': nan}],
 'auto_normalizado': {'marca': 'MITSUBISHI',
  'modelo': 'LANCER',
  'cilindrada': '1.6',
  'año': 2010,
  'version': ''}}

In [175]:
verificar_auto_nuevo(
    {'marca': 'Hyundai', 'modelo': 'Accent', 'año': 2013},
    df_marca, df_modelo, df_autos
)


{'existe': False,
 'mensaje': 'Auto no encontrado exactamente, pero hay modelos similares',
 'coincidencias': [{'MARCA': 'HYUNDAI',
   'MODELO': 'ACCENT',
   'CILINDRADA': '1.7',
   'AÑO': 2011.0,
   'VERSION': nan},
  {'MARCA': 'HYUNDAI',
   'MODELO': 'ACCENT',
   'CILINDRADA': '1.6',
   'AÑO': 2006.0,
   'VERSION': nan},
  {'MARCA': 'HYUNDAI',
   'MODELO': 'ACCENT',
   'CILINDRADA': '1.6',
   'AÑO': 2007.0,
   'VERSION': nan},
  {'MARCA': 'HYUNDAI',
   'MODELO': 'ACCENT',
   'CILINDRADA': '1.6',
   'AÑO': 2008.0,
   'VERSION': nan},
  {'MARCA': 'HYUNDAI',
   'MODELO': 'ACCENT',
   'CILINDRADA': '1.6',
   'AÑO': 2009.0,
   'VERSION': nan}],
 'auto_normalizado': {'marca': 'HYUNDAI',
  'modelo': 'ACCENT',
  'cilindrada': '',
  'año': 2013,
  'version': ''}}

In [176]:
auto = {'marca': 'Toyota', 'modelo': 'Corolla', 'cilindrada': '1.6', 'año': 2005}
resultado = verificar_auto_nuevo(auto, df_marca, df_modelo, df_autos)
resultado

{'existe': True,
 'mensaje': 'Auto existe en la base de datos',
 'coincidencias': [{'MARCA': 'TOYOTA',
   'MODELO': 'COROLLA',
   'CILINDRADA': '1.6',
   'AÑO': 2005.0,
   'VERSION': nan}],
 'auto_normalizado': {'marca': 'TOYOTA',
  'modelo': 'COROLLA',
  'cilindrada': '1.6',
  'año': 2005,
  'version': ''}}